In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

In [2]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform, download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 82931157.33it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 27227622.80it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 19972539.92it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3999691.11it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
# Define data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# Define a wider and deeper neural network model
class WiderDeeperNet(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(WiderDeeperNet, self).__init__()
        self.layers = nn.ModuleList()
        prev_size = input_size

        # Create a wider and deeper architecture
        for hidden_size in hidden_sizes:
            self.layers.append(nn.Linear(prev_size, hidden_size))
            self.layers.append(nn.ReLU())
            prev_size = hidden_size
        self.fc_out = nn.Linear(prev_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        for layer in self.layers:
            x = layer(x)
        out = self.fc_out(x)
        return out

In [5]:
# Function to train and evaluate the model
def train_and_evaluate(model, train_loader, test_loader, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []

        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels, average='weighted')
        precision = precision_score(true_labels, predicted_labels, average='weighted')
        recall = recall_score(true_labels, predicted_labels, average='weighted')

    return accuracy, f1, precision, recall

In [6]:
# Define three different model configurations
hidden_sizes_config1 = [128, 64]  # Wider and deeper
hidden_sizes_config2 = [256, 128, 64]  # Wider and deeper
hidden_sizes_config3 = [64, 32]  # Shallower and narrower

# Train and evaluate each configuration
model1 = WiderDeeperNet(input_size=28*28, hidden_sizes=hidden_sizes_config1, num_classes=10)
accuracy1, f1_1, precision1, recall1 = train_and_evaluate(model1, train_loader, test_loader)

model2 = WiderDeeperNet(input_size=28*28, hidden_sizes=hidden_sizes_config2, num_classes=10)
accuracy2, f1_2, precision2, recall2 = train_and_evaluate(model2, train_loader, test_loader)

model3 = WiderDeeperNet(input_size=28*28, hidden_sizes=hidden_sizes_config3, num_classes=10)
accuracy3, f1_3, precision3, recall3 = train_and_evaluate(model3, train_loader, test_loader)

# Print and explain the performance results
print("Performance Results:")
print(f"Configuration 1: Wider and Deeper Model - Accuracy: {accuracy1:.4f}, F1 Score: {f1_1:.4f}, Precision: {precision1:.4f}, Recall: {recall1:.4f}")
print(f"Configuration 2: Wider and Deeper Model - Accuracy: {accuracy2:.4f}, F1 Score: {f1_2:.4f}, Precision: {precision2:.4f}, Recall: {recall2:.4f}")
print(f"Configuration 3: Shallower and Narrower Model - Accuracy: {accuracy3:.4f}, F1 Score: {f1_3:.4f}, Precision: {precision3:.4f}, Recall: {recall3:.4f}")

Performance Results:
Configuration 1: Wider and Deeper Model - Accuracy: 0.9662, F1 Score: 0.9663, Precision: 0.9673, Recall: 0.9662
Configuration 2: Wider and Deeper Model - Accuracy: 0.9781, F1 Score: 0.9781, Precision: 0.9782, Recall: 0.9781
Configuration 3: Shallower and Narrower Model - Accuracy: 0.9693, F1 Score: 0.9693, Precision: 0.9695, Recall: 0.9693


####Result:

Dari hasil performance results menggunakan dataset ini dapat disimpulkan bahwa configurasi dengan skala yang lebih besar dan dalam akan mendapatkan nilai akurasi yang lebih tinggi dibandingkan dengan skala yang lebih kecil dan *not deeper*. Bisa di lihat jika code di eksekusi Configuration 2 lah yang mendapatkan nilai Accuracy, F1 Score, Precision, Recall lebih besar dibandingkan yang lainnya.